In [23]:
import logging
import pandas as pd
import os
import json

In [24]:
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery(sessionid="72306954009%3AgwJxLcXKJdgNcO%3A14%3AAYd34XjCSKW5z66yEcKtd_IszQQmcZ9i5rhqGKPfwg")
# t = RetrieveDataByQuery(sessionid="67725660272%3AW741uU0WEqnpeU%3A11%3AAYc8uSJT5zhhAmFGrTsolApwMbEuOfyAr5P4hqX8IA")
# t = RetrieveDataByQuery(sessionid="72311449442%3AlUryE0KV76UBLs%3A27%3AAYcrmho9upEOXJN1UXfXFAcPBrxEthfQark1yarqDg")
# t = RetrieveDataByQuery(sessionid="72164089558%3ArAwUfcPDA9EFBe%3A11%3AAYfCrETIzCPGInwfJ21wGhSlRwbbdWTecG10Kd5GaA")
# t = RetrieveDataByQuery(sessionid="72164089558%3As83zYF4EnDh0UB%3A9%3AAYeSZtx6U69boU7q1yS7sXUdDgCwWuzZP-LYlG2hDg")
# t = RetrieveDataByQuery(sessionid="72308001558%3A2178PDc0NJh5Q0%3A19%3AAYdWYk73GG4xii7piBJ3BrZlGjR706FAxvwYY1O2HA")
# t.retrieve_follower_by_userid_after("62231305945", str(25 * 1))

In [25]:
# query = "Artificial Intelligence"

In [26]:
def get_follower_data(data: list):
    try:
        return [
            {
                "pk": node.get("pk", ""),
                "username": node.get("username", ""),
                "follower_count": node.get("follower_count", "")
            }
            for d in data
            if (node := d.get("node", {}))
        ]
    except Exception as e:
        raise RuntimeError(f"Errore durante l'elaborazione dei dati dei follower: {e}")


progress_tracker = {}

def save_progress(userid: str, batch_number: int, results: list):
    """Salva il batch_number corrente e i risultati ottenuti per un determinato userid."""
    progress_tracker[userid] = {"batch_number": batch_number, "results": results}
    with open("progress2.json", "w") as f:
        json.dump(progress_tracker, f)

def load_progress(userid: str):
    """Carica il batch_number e i risultati salvati per un determinato userid."""
    try:
        with open("progress2.json", "r") as f:
            data = json.load(f)
            return data.get(userid, {"batch_number": 1, "results": []})
    except (FileNotFoundError, json.JSONDecodeError):
        return {"batch_number": 1, "results": []}
def remove_progress(userid: str):
    """Rimuove il progresso salvato per un determinato userid."""
    try:
        with open("progress2.json", "r") as f:
            data = json.load(f)
        
        if userid in data:
            del data[userid]
            with open("progress2.json", "w") as f:
                json.dump(data, f)
    except (FileNotFoundError, json.JSONDecodeError):
        pass
    
def get_all_follower(userid: str):
    progress = load_progress(userid)
    batch_number = progress["batch_number"]
    results = progress["results"]
    allpk = [res["pk"] for res in results]
    try:
        if batch_number == 1:
            response = t.retrieve_follower_by_userid(userid)
            data = response.get("data", {}).get("user", {}).get("followers", {})
            d_out = get_follower_data(data.get("edges", []))
            for res in d_out:
                allpk.append(res["pk"])
            results.extend(d_out)
            has_next_page = data.get("page_info", {}).get("has_next_page", False)
        else:
            has_next_page = True

        while has_next_page:
            try:
                print("Pagina:", batch_number)
                response = t.retrieve_follower_by_userid_after(userid, str(25 * batch_number))
                batch_number += 1
                data = response.get("data", {}).get("fetch__XDTUserDict", {}).get("followers", {})
                d_out = get_follower_data(data.get("edges", []))
                for res in d_out:
                    if res["pk"] in allpk:
                        remove_progress(userid)
                        return results
                    allpk.append(res["pk"])
                results.extend(d_out)
                
                has_next_page = data.get("page_info", {}).get("has_next_page", False)
                
                # Salva il progresso dopo ogni batch
                save_progress(userid, batch_number, results)
                
            except Exception as e:
                save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore
                raise RuntimeError(f"Errore nel recupero della paginazione dei follower: {e}")
    
    except Exception as e:
        save_progress(userid, batch_number, results)  # Salva il batch e i risultati in caso di errore generale
        raise RuntimeError(f"Errore generale nel recupero dei follower per l'utente {userid}: {e}")
    remove_progress(userid)
    return results




def get_post_data(posts: list):
    results = []
    for post in posts:
        dati = post["node"]["thread"]["thread_items"]
        for p in dati:
            d = p['post']
            if int(d.get("like_count", 0)) < 100:
                continue
            userpk = d.get("user", {}).get("pk", "")
            caption = d.get("caption", {})
            caption_text = caption.get("text", "") if caption else ""
            if caption_text == "":
                continue
            results.append({
                "pk": d.get("pk", ""),
                "user_pk": userpk,
                "caption": caption_text,
                "like_count": int(d.get("like_count", 0)),
                "taken_at": d.get("taken_at", "")
            })
    return results

In [27]:


# results = []
# response = t.retrieve_thread_by_query(query)
# data = response.get("data", {}).get("searchResults", {})
# results.extend(get_post_data(data.get("edges", [])))
# has_next_page = data.get("page_info", {}).get("has_next_page", False)
# after = data.get("page_info", {}).get("end_cursor")
# i = 1000

# while has_next_page and i > 0:
#     response = t.retrieve_thread_by_query_after(query, after)
#     data = response.get("data", {}).get("searchResults", {})
#     results.extend(get_post_data(data.get("edges", [])))
#     has_next_page = data.get("page_info", {}).get("has_next_page", False)
#     after = data.get("page_info", {}).get("end_cursor")
#     i -= 1

# # Creazione del DataFrame
# df = pd.DataFrame(results)
# df.shape

In [28]:
# df.drop_duplicates(inplace=True)
# df['taken_at'] = pd.to_datetime(df['taken_at'], unit='s')
# df.to_csv("data.csv", index=False)

In [29]:
df = pd.read_csv("sottofile_5.csv")
df['follower_pk'] = df['follower_pk'].astype(str)
unique_user_pk = df['follower_pk'].unique()
len(unique_user_pk)

61121

In [30]:
def load_processed_users(file_path):
    if os.path.exists(file_path):
        logging.info(f"Loading processed users from {file_path}")
        df = pd.read_csv(file_path)
        df['user_pk'] = df['user_pk'].astype(str)
        return set(df['user_pk'].tolist())
    return set()

def save_to_csv(df, file_path):
    try:
        if os.path.exists(file_path):
            df.to_csv(file_path, mode='a', header=False, index=False)
        else:
            df.to_csv(file_path, index=False)
        logging.info(f"Saved {len(df)} records to {file_path}")
    except Exception as e:
        logging.error(f"Error saving data to {file_path}: {e}")
        
csv_file = "followers_followers_Artificial_Intelligence5.csv"
processed_users = load_processed_users(csv_file)

In [31]:
unique_user_pk = unique_user_pk[unique_user_pk!="55652026084"]

In [32]:
len(processed_users)

0

In [33]:
def fetch_followers(pk):
    print(f"Fetching followers for user_pk: {pk}")
    try:
        follower_data = get_all_follower(pk)
        return [
            {
                "user_pk": pk,
                "follower_pk": follower["pk"],
                "follower_username": follower["username"],
                "follower_count": follower["follower_count"]
            }
            for follower in follower_data
        ]
    except Exception as e:
        logging.error(f"Error fetching followers for {pk}: {e}")
        return None



follower_list = []
batch_size = 1


print("Starting follower data collection")
for user_pk in unique_user_pk:
    if user_pk in processed_users:
        print(f"Skipping already processed user_pk: {user_pk}")
        continue
    
    processed_users.update([user_pk])
    results = fetch_followers(user_pk)
    if results is None:
        break
    follower_list.extend(results)
    
    if follower_list:
        print(f"Processing batch of {len(follower_list)} followers")
        follower_df = pd.DataFrame(follower_list)
        save_to_csv(follower_df, csv_file)
        follower_list = []

if follower_list:
    print(f"Saving remaining {len(follower_list)} followers")
    follower_df = pd.DataFrame(follower_list)
    save_to_csv(follower_df, csv_file)

print("Follower data collection completed")

Starting follower data collection
Fetching followers for user_pk: 6036769896


Fetching followers for user_pk: 2524381541
Fetching followers for user_pk: 4109120461
Processing batch of 16 followers
Fetching followers for user_pk: 67121990082
Fetching followers for user_pk: 2874984984
Fetching followers for user_pk: 32792905020
Processing batch of 6 followers
Fetching followers for user_pk: 6927195057
Pagina: 1
Pagina: 2
Processing batch of 62 followers
Fetching followers for user_pk: 62226325024
Pagina: 1
Processing batch of 29 followers
Fetching followers for user_pk: 12575146596
Processing batch of 1 followers
Fetching followers for user_pk: 4569853744
Fetching followers for user_pk: 65213343012
Pagina: 1
Processing batch of 27 followers
Fetching followers for user_pk: 3244054153
Fetching followers for user_pk: 45537611527
Pagina: 1
Processing batch of 29 followers
Fetching followers for user_pk: 6020225083
Processing batch of 13 followers
Fetching followers for user_pk: 68916682791
Processing batch of 24 followers
Fetching followers for user_pk: 4466073050
Pag

ERROR:root:Error fetching followers for 55147590135: Errore generale nel recupero dei follower per l'utente 55147590135: Expecting value: line 1 column 1 (char 0)


Follower data collection completed


In [34]:
df1 = pd.read_csv(csv_file)
df_count= df1.drop_duplicates().groupby("user_pk").count()
print(df_count.shape)
df_count

(15, 3)


,follower_pk,follower_username,follower_count
user_pk,,,
4109120461,16,16,16
4466073050,25,25,25
6020225083,13,13,13
6927195057,62,62,62
8312794358,2,2,2
12575146596,1,1,1
14814262214,25,25,25
32792905020,6,6,6
45537611527,29,29,29


In [35]:
df1 = pd.read_csv("followers_data2.csv")
df1 = df1.drop_duplicates()
df1
df1.to_csv("followers_data2.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'followers_data2.csv'